# Import packages

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from common import *

# 1. Load dataset

In [2]:
train_data = pd.read_csv(join(PATH.input, "train_data.csv"))
sample_submission = pd.read_csv(join(PATH.input, "sample_submission.csv"))
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3859200 entries, 0 to 3859199
Data columns (total 13 columns):
 #   Column   Dtype  
---  ------   -----  
 0   TurbID   int64  
 1   Day      int64  
 2   Tmstamp  object 
 3   Wspd     float64
 4   Wdir     float64
 5   Etmp     float64
 6   Itmp     float64
 7   Ndir     float64
 8   Pab1     float64
 9   Pab2     float64
 10  Pab3     float64
 11  Prtv     float64
 12  Patv     float64
dtypes: float64(10), int64(2), object(1)
memory usage: 382.8+ MB


# 2. Preprocessing

## 1) Imputing

In [3]:
train_data = impute_data(train_data)

Number of Nan values: 0


## 2) Convert dtype of `Tmstamp`

In [4]:
tms_list = list(pd.unique(train_data['Tmstamp']))
train_data['Tmstamp'] = train_data['Tmstamp'].apply(lambda x: tms_list.index(x)+1)

## 3) Split dataset

In [30]:
SEQ_LEN = 5
(train_x, train_y), (val_x, val_y), (test_x, test_y) = make_train_val_test_data(train_data, SEQ_LEN)

100%|██████████| 134/134 [00:25<00:00,  5.35it/s]


Train data(X, y): (24522, 720, 11) (24522, 288)
Validation data(X, y): (134, 720, 11) (134, 288)
Test data(X, y): (134, 720, 11) (0,)


# 3. Modeling

In [31]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU

tf.keras.utils.set_random_seed(RANDOM_STATE)
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

2022-07-11 13:40:07.065733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 13:40:07.069225: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 13:40:07.069393: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [32]:
model = Sequential([
    GRU(256, input_shape=train_x[0].shape),
    Dense(516, activation='relu'),
    Dense(288, activation='relu')
])
optimizer = tf.optimizers.RMSprop(0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

2022-07-11 13:40:09.134059: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-11 13:40:09.134819: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 13:40:09.135090: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 13:40:09.135277: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

# 4. Training

In [33]:
from tensorflow.keras.callbacks import EarlyStopping

model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=1000, batch_size=128, callbacks=[EarlyStopping(patience=10, restore_best_weights=True)]);

Epoch 1/1000


2022-07-11 13:40:16.489603: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


  3/192 [..............................] - ETA: 6s - loss: 324457.5938 - mae: 372.1783  

2022-07-11 13:40:17.014986: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


192/192 [==============================] - 9s 38ms/step - loss: 186423.5156 - mae: 327.6572 - val_loss: 306593.9688 - val_mae: 469.6978
Epoch 2/1000
192/192 [==============================] - 7s 37ms/step - loss: 161837.2969 - mae: 312.5818 - val_loss: 247971.4219 - val_mae: 412.7185
Epoch 3/1000
192/192 [==============================] - 7s 37ms/step - loss: 151041.7344 - mae: 300.1900 - val_loss: 246048.4219 - val_mae: 416.4329
Epoch 4/1000
192/192 [==============================] - 7s 37ms/step - loss: 143353.3750 - mae: 290.3304 - val_loss: 263508.0000 - val_mae: 449.9444
Epoch 5/1000
192/192 [==============================] - 7s 37ms/step - loss: 137013.2500 - mae: 281.7166 - val_loss: 227419.7812 - val_mae: 403.8564
Epoch 6/1000
192/192 [==============================] - 7s 37ms/step - loss: 131155.4688 - mae: 273.8357 - val_loss: 215563.3750 - val_mae: 379.5534
Epoch 7/1000
192/192 [==============================] - 7s 37ms/step - loss: 126386.5312 - mae: 267.2465 - val_loss: 20

# 6. Generate submission

In [35]:
sample_submission['Patv'] = model.predict(test_x).reshape(-1)
sample_submission.to_csv(join(PATH.output, "proposed5.csv"), index=False)

5/5 [==============================] - 0s 13ms/step
